In [ ]:
from __future__ import print_function, division, absolute_import

# %load_ext autoreload
# %autoreload 2

import safeopt
import GPy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets # Widget definitions
from IPython import display # Used to display widgets in the notebook

## Define a kernel and function

Here we define a kernel. The function is drawn at random from the GP and is corrupted my Gaussian noise

In [ ]:
# Measurement noise
noise_var = 0.05 ** 2

# Set fixed Gaussian measurement noise
likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)

# Bounds on the inputs variable
bounds = [(-10., 10.)]

# Define Kernel
kernel = GPy.kern.RBF(input_dim=len(bounds), variance=2., lengthscale=1.0, ARD=True)

# set of parameters
parameter_set = safeopt.linearly_spaced_combinations(bounds, 1000)

# Initial safe point
x0 = np.zeros((1, len(bounds)))

# Generate function with safe initial point at x=0
def sample_safe_fun():
    while True:
        fun = safeopt.sample_gp_function(kernel, bounds, noise_var, 100)
        if fun(0, noise=False) > 0.5:
            break
    return fun

## Interactive run of the algorithm

In [ ]:
button_add = widgets.Button(description='Add sample', tooltip='Sample a new data point for the optimization')
button_reset = widgets.Button(description='Reset', tooltip='Restart the safe optimization algorithm')
button_new = widgets.Button(description='New function', tooltip='Add a new function and reset')
slider_safety = widgets.FloatSlider(description='Safety threshold', value=0, min=-4, max=4, step=0.1)

ms = 20
mew = 7
lw = 5

def plot_gp():
    gp_opt.plot(figure=plt.figure(figsize=(15, 8)), lw=lw)
    
    # Plot last point red
    if gp_opt.gp is not None:
        plt.plot(gp_opt.gp.X[-1, :], gp_opt.gp.Y[-1, :],
                 'rx', ms=ms, mew=mew, label='Last Point')
        
    # Plot safe line
    if len(bounds) == 1:
        plt.plot([bounds[0][0], bounds[0][1]], [gp_opt.fmin, gp_opt.fmin],
                 'k--', label='Minimum', lw=lw)
        
    plt.plot(true_values[0], true_values[1],
             alpha=0.15, label='True function', lw=lw)
    
    plt.ylim([-4, 4])
    plt.legend(loc=3)
    
    # Ensure we only get one plot
    display.clear_output(wait=True)

def new_sample(b=None):
    """Draw a new gp sample"""
    gp_opt.optimize()
    plot_gp()
button_add.on_click(new_sample)

def reset(b=None):
    """Reset the GP-UCB algorithm"""
    global gp_opt
    gp = GPy.core.GP(x0, fun(x0), kernel, likelihood)
    gp_opt = safeopt.SafeOpt(fun, gp, parameter_set, 0.)
    plot_gp()
button_reset.on_click(reset)

def new_fun(b=None):
    """Draw a new function from the GP"""
    global fun
    global true_values
    fun = sample_safe_fun()
    inp = np.linspace(bounds[0][0], bounds[0][1], 200)[:, None]
    true_values = (inp, fun(inp, noise=False))
    reset(b)
button_new.on_click(new_fun)

def change_safety(b=None):
    if b == 'value':
        gp_opt.fmin = slider_safety.get_state()['value']
        plot_gp()
slider_safety.on_trait_change(change_safety)

display.display(button_add, button_new, button_reset, slider_safety)
new_fun()